In [1]:
# importing all the necessary libaries
import os
import csv
import time
import string
import keyboard
from re import search

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support import expected_conditions as EC

In [2]:
# define your school name right here:

School_name = "Institut National des Postes et Télécommunications"

In [14]:
# defining the webdriver and config btw this code will be almost the same in all of your selenium scripts
options = Options()

# !!! blocking browser notifications !!!
prefs = {"profile.default_content_setting_values.notifications": 2}
options.add_experimental_option("prefs", prefs)

# starting in maximized window
options.add_argument("start-maximized")
options.add_argument("--disable-default-apps")
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)



====== WebDriver manager ======
Current google-chrome version is 99.0.4844
Get LATEST chromedriver version for 99.0.4844 google-chrome
Driver [C:\Users\taha_\.wdm\drivers\chromedriver\win32\99.0.4844.51\chromedriver.exe] found in cache


In [15]:
driver.get("https://www.linkedin.com/login/")

In [16]:
username = os.environ.get("ln_us")
password = os.environ.get("ln_ps")

In [17]:
username_field = driver.find_element(By.ID,"username")
username_field.send_keys(username)

password_field = driver.find_element(By.ID,"password")
password_field.send_keys(password)

password_field.submit()

In [18]:
'''search_bar = driver.find_element(By.CLASS_NAME,"search-global-typeahead__input")
search_bar.send_keys(School_name)
time.sleep(1)
keyboard.press_and_release('enter')'''

'search_bar = driver.find_element(By.CLASS_NAME,"search-global-typeahead__input")\nsearch_bar.send_keys(School_name)\ntime.sleep(1)\nkeyboard.press_and_release(\'enter\')'

In [19]:
driver.get("https://www.linkedin.com/search/results/schools/?keywords="+School_name)


In [20]:
time.sleep(2)
first_result = WebDriverWait(driver, 2).until(EC.element_to_be_clickable((By.XPATH, """//*[@id="main"]/div/div/div[1]/ul/li/div/div/div[2]/div/div[1]/div/span/span/a""")))
first_result.click()

In [21]:
driver.get(driver.current_url+"people")

In [22]:
count = 0
items_urls = []
testing = []
cards_links = driver.find_elements(By.CLASS_NAME, "org-people-profile-card__profile-info")
print(len(cards_links))
for card in cards_links: print(card.text+"\n----------------------------------\n")


0


In [23]:
while len(cards_links) < 100:

    # keyboard.press_and_release('down arrow')
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(.7)
    try:
        show_more = WebDriverWait(driver, 1).until(EC.element_to_be_clickable((By.XPATH, "//*[contains(text(), 'Show more results')]")))
        show_more.click()
    except:
        continue
    cards_links = driver.find_elements(By.CLASS_NAME, "org-people-profile-card__profile-info")
    print(len(cards_links))

KeyboardInterrupt: 

In [24]:
print("lengh is: "+str(len(cards_links)))
for card in cards_links: print(card.text+"\n----------------------------------\n")

lengh is: 0


In [ ]:

count = 0
items_urls = []
testing = []
while len(items_urls) < 100:

    # keyboard.press_and_release('down arrow')
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight/.1);")
    time.sleep(.7)
    cards_links = driver.find_elements(By.CLASS_NAME, "org-people-profile-card__profile-info")
    if cards_links == testing:
        break
    for card in cards_links:
        try:
            item_url = card.get_attribute('href')
            items_urls.append(item_url)
        except:
            continue
    testing = driver.find_elements(By.CLASS_NAME, "org-people-profile-card__profile-info")
    cards_links = []
    print("lengh is: "+str(len(testing)))
    for card in testing: print(card.text+"\n----------------------------------\n")


In [ ]:
while len(items_urls) < 100:

    # keyboard.press_and_release('down arrow')
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight/.1);")
    time.sleep(.7)
    cards_links = driver.find_elements(By.CLASS_NAME, "org-people-profile-card__profile-info")
    if cards_links == testing:
        count += 1
    if count == 10:
        break
    for card in cards_links:
        try:
            item_url = card.get_attribute('href')
            items_urls.append(item_url)
        except:
            continue
    testing = driver.find_elements_by_xpath(
        "//section[@class='annonces ng-star-inserted']//a")
    cards_links = []
    print(len(items_urls))